<a href="https://colab.research.google.com/github/manika137/AI-ML-Month/blob/main/Ukraine_Russia_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# utilities
import re
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# plotting
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

# nltk
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer

# important libraries
#from bokeh.plotting import figure
#from bokeh.io import output_file, show, output_notebook
#from collections import Counter
#from spacy.util import minibatch
#from spacy import displacy
#import gc
import os

In [ ]:
all_files = []
for dirname, _, filenames in os.walk('/content/'):
    for filename in filenames:
        full_path=os.path.join(dirname, filename)
        all_files.append(full_path)

In [ ]:
all_files.sort()

In [ ]:
all_files

['/content/.config/.last_opt_in_prompt.yaml',
 '/content/.config/.last_survey_prompt.yaml',
 '/content/.config/.last_update_check.json',
 '/content/.config/active_config',
 '/content/.config/config_sentinel',
 '/content/.config/configurations/config_default',
 '/content/.config/gce',
 '/content/.config/logs/2022.07.13/13.41.43.930690.log',
 '/content/.config/logs/2022.07.13/13.42.05.105178.log',
 '/content/.config/logs/2022.07.13/13.42.25.014838.log',
 '/content/.config/logs/2022.07.13/13.42.32.441281.log',
 '/content/.config/logs/2022.07.13/13.42.53.106562.log',
 '/content/.config/logs/2022.07.13/13.42.53.841315.log',
 '/content/sample_data/README.md',
 '/content/sample_data/anscombe.json',
 '/content/sample_data/california_housing_test.csv',
 '/content/sample_data/california_housing_train.csv',
 '/content/sample_data/mnist_test.csv',
 '/content/sample_data/mnist_train_small.csv']

In [ ]:
apr_files = [file for file in all_files if re.search(r"04+", file)]
apr_files


[]

In [ ]:
june = [file for file in all_files if re.search(r"05+", file) and re.search(r"Ukraine+", file)]
june


['/content/0501_UkraineCombinedTweetsDeduped.csv.gzip',
 '/content/0502_UkraineCombinedTweetsDeduped.csv.gzip']

In [ ]:
data = []
for file in june:
    print(f"Reading in {file}")
    # unzip and read in the csv file as a dataframe
    tmp_df = pd.read_csv(file, compression="gzip", header=0, index_col=0)
    # append dataframe to temp list
    data.append(tmp_df)
    

#print("Concatenating the DataFrames")
# concatenate the dataframes in the temp list row-wise
#apr_files= pd.concat(data, axis=0) 

Reading in /content/0501_UkraineCombinedTweetsDeduped.csv.gzip
Reading in /content/0502_UkraineCombinedTweetsDeduped.csv.gzip


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df = pd.DataFrame(data, dtype=object)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [ ]:
df.head()

,0
0,userid username ...
1,userid username ...


In [ ]:
from chardet import detect

# get file encoding type
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

In [ ]:
import gzip
import csv

f = gzip.open('0401_UkraineCombinedTweetsDeduped.csv.gzip', mode="rt")
csvobj = csv.reader(f,delimiter = ',',quotechar="'")

data = []
i = 1
for rows in csvobj:
  if i == 10:
    break
  data.append(rows)
  print(rows)


Streaming output truncated to the last 5000 lines.
['#UkraineRussiaWar #USMNT @FBIWFO #UkraineWar #Ukraine https://t.co/YK9xGRoMRP"', '"[{\'text\': \'UkraineRussiaWar\'', " 'indices': [223", ' 240]}', " {'text': 'USMNT'", " 'indices': [241", ' 247]}', " {'text': 'UkraineWar'", " 'indices': [256", ' 267]}', " {'text': 'Ukraine'", " 'indices': [268", ' 276]}]"', '"en"', '""', '"0"', '"2022-04-01 00:24:31.718372"']
['"2022"', '"1102531512560439296"', '"ElonMuskFr"', '"@elonmusk', ' en français 🚀 #TeamMuskitos \uea00']
[]
['Compte sérieux et non parodique n’appartenant pas au principal intéressé."', '"France"', '"60"', '"29544"', '"2790"', '"2019-03-04 11:29:31.000000"', '"1509684521960157186"', '"2022-04-01 00:10:13.000000"', '"0"', '"VIDEO. Guerre en #Ukraine : comment la société d\'#ElonMusk (@elonmusk) aide-t-elle les drones ukrainiens ? 🌍 [franceinfo] https://t.co/fu5KgzHsBd"', '"[{\'text\': \'Ukraine\'', " 'indices': [17", ' 25]}', " {'text': 'ElonMusk'", " 'indices': [49", ' 58]}]"'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tmp_df_list = []
for file in june:
    print(f"Reading in {file}")

    # unzip and read in the csv file as a dataframe
    tmp_df = pd.read_csv(file, compression="gzip", engine='python', header = 0, index_col = 0)

    # append dataframe to temp list
    tmp_df_list.append(tmp_df)

print("Concatenating the DataFrames")
# concatenate the dataframes in the temp list row-wise

june = pd.concat(tmp_df_list, axis=0)
print("Concatenation complete!")


Reading in /content/0501_UkraineCombinedTweetsDeduped.csv.gzip
Reading in /content/0502_UkraineCombinedTweetsDeduped.csv.gzip
Concatenating the DataFrames
Concatenation complete!


In [ ]:
df

,0
0,userid username ...
1,userid username ...


In [ ]:
from subprocess import check_output
print(check_output(["ls", "/content"]).decode("utf8"))

In [ ]:
gzip_df_small = pd.read_csv('/content/0401_UkraineCombinedTweetsDeduped.csv.gzip', compression='gzip', engine='python', header=0, sep=',', quotechar='"')
gzip_df_small.head(10)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import nltk
from nltk.corpus import stopwords
import spacy
from spacy.util import compounding
from spacy.util import minibatch
from spacy import displacy
import gc
import os

In [ ]:
csv_collection = []
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        fullpath = os.path.join(dirname, filename)
        csv_collection.append(fullpath)

In [ ]:
csv_collection

['/content/0502_UkraineCombinedTweetsDeduped.csv.gzip',
 '/content/0428_UkraineCombinedTweetsDeduped.csv.gzip',
 '/content/0430_UkraineCombinedTweetsDeduped.csv.gzip',
 '/content/0401_UkraineCombinedTweetsDeduped.csv.gzip',
 '/content/0501_UkraineCombinedTweetsDeduped.csv.gzip',
 '/content/.config/active_config',
 '/content/.config/.last_update_check.json',
 '/content/.config/gce',
 '/content/.config/.last_survey_prompt.yaml',
 '/content/.config/.last_opt_in_prompt.yaml',
 '/content/.config/config_sentinel',
 '/content/.config/logs/2022.07.13/13.42.32.441281.log',
 '/content/.config/logs/2022.07.13/13.41.43.930690.log',
 '/content/.config/logs/2022.07.13/13.42.53.106562.log',
 '/content/.config/logs/2022.07.13/13.42.53.841315.log',
 '/content/.config/logs/2022.07.13/13.42.05.105178.log',
 '/content/.config/logs/2022.07.13/13.42.25.014838.log',
 '/content/.config/configurations/config_default',
 '/content/sample_data/anscombe.json',
 '/content/sample_data/README.md',
 '/content/sample_d

In [ ]:
apr_files = [file[9:] for file in csv_collection if re.search(r"04+", file)]
apr_files

['0428_UkraineCombinedTweetsDeduped.csv.gzip',
 '0430_UkraineCombinedTweetsDeduped.csv.gzip',
 '0401_UkraineCombinedTweetsDeduped.csv.gzip']

In [ ]:
tmp_df_list = []

for file in apr_files:
    print(f"Reading in {file}")

    # unzip and read in the csv file as a dataframe
    tmp_df = pd.read_csv(file, compression="gzip", header=0, index_col=0)
    
    # append dataframe to temp list
    tmp_df_list.append(tmp_df)


Reading in 0428_UkraineCombinedTweetsDeduped.csv.gzip
Reading in 0430_UkraineCombinedTweetsDeduped.csv.gzip


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Reading in 0401_UkraineCombinedTweetsDeduped.csv.gzip


UnicodeDecodeError: ignored

In [ ]:
df = pd.read_csv(csv_collection.pop(), compression = 'gzip', index_col = 0)

for data in csv_collection:
    try:
        tmp = pd.read_csv(data, compression = 'gzip', index_col = 0)
    except:
        tmp = pd.read_csv(data, index_col = 0)
    df = pd.concat([df, tmp], axis=0)
    

OSError: ignored

In [ ]:
os.getcwd()

'/content'